# Labolatorium 2

In [50]:
import gmpy2
import random
import time
from gmpy2 import mpz

def random_elem_zn(k):
    random.seed(int(time.time()))
    bin = "1" +"".join(str(random.randint(0,1)) for bit in range(k-1))
    r = mpz(bin, 2)
    return r

def random_prime(k, p):
    r = gmpy2.mpz(random_elem_zn(k=k))
    while True:
        r = gmpy2.next_prime(r)
        if r % gmpy2.mpz(4) == p:
            return r
        
def effective_power(b,k,n):
    resoult = 1
    b = b % n
    while k>0:
        if k % 2 == 1:
            resoult = (resoult * b) % n
            
        k = k//2
        b = (b * b) % n
    return resoult

def is_quadratic_residue(b, p):
    resoult = gmpy2.powmod(b, (p-1)//2, p)
    return resoult == 1

def sqrt_mod_p(b, p):
    resoult = effective_power(b=b, k=(p+1)//4, n=p)
    return resoult

## Zadanie 1
Zaimplementuj algorytm (funkcję), która generuje losową krzywą eliptyczną nad F<sub>p</sub>. <br>
Dane: p = 3 (mod 4) duża liczba pierwsza (ok. 300 bitów) <br>
Wynik: A, B ∈ Fp takie, że E : Y <sup>2</sup> = X<sup>3</sup> + AX + B jest krzywą nad F<sub>p</sub>

In [51]:
def generate_random_curve(p):
    rand_state = gmpy2.random_state(int(time.time()))
    while True:
        A = gmpy2.mpz_random(rand_state, p)
        B = gmpy2.mpz_random(rand_state, p)
        if (4 * gmpy2.powmod(A, 3, p) + 27 * gmpy2.powmod(B, 2, p)) % p != 0:
            return A, B


p = random_prime(300,3)
A, B = generate_random_curve(p=p)
print(f"P:  {p}\n A:  {A}\n B:  {B}\n")

P:  1732080215712002454704681355387155171099538896304905967017952161440184063745826556907903731
 A:  1438505434365605582005427352651579320277270490020430950880932510659014446801249121190807032
 B:  828572917202432995858406360376239070110672729658239655166938974483622242735504065560901860



## Zadanie 2
Zaimplementuj algorytm (funkcję), który znajduje losowy punkt na krzywej eliptycznej nad F<sub>p</sub>. <br>
Dane: A, B, p = 3 (mod 4) takie, że E : Y<sup>2</sup> = X<sup>3</sup> + AX + B jest krzywą nad Fp<br>
Wynik: P = (x, y) ∈ E(F<sub>p</sub>)

In [52]:
def find_point_on_curve(A,B,p):
    rand_state = gmpy2.random_state(int(time.time()))

    while True:
        x = gmpy2.mpz_random(rand_state,p)
        y_pow_2 = ( effective_power(x,3,p) + A*x + B ) % p
        if is_quadratic_residue(y_pow_2, p):
            y = sqrt_mod_p(y_pow_2, p)
            return x,y
        

p = random_prime(k=300,p=3)
A, B = generate_random_curve(p=p)
x, y = find_point_on_curve(A,B,p)
print(f"P=({x},{y})")
x, y = find_point_on_curve(1, 1, 7)
print(f"P=({x},{y})")

        


P=(632001240931913206424725296092737555291996466411769017127257663531360159738505913372800853,343089392046902625866744035273609795661765790634797233489747353499555070021147516944676200)
P=(0,1)


## Zadanie 3
Zaimplementuj algorytm (funkcję), który oblicza punkt przeciwny do danego punktu. <br>
Dane: P = (x, y) ∈ E(F<sub>p</sub>) <br>
Wynik: −P = (x, −y) ∈ E(F<sub>p</sub>) <br>

In [53]:
def opposite_point(x,y,p):
    o_y = (-y) % p
    o_x = x
    return o_x, o_y


p = random_prime(k=300,p=3)
A, B = generate_random_curve(p=p)
x, y = find_point_on_curve(A,B,p)
o_x, o_y = opposite_point(x,y,p)
print(f"-P=({o_x},{o_y})")

o_x, o_y = opposite_point(2,5,7)
print(f"-P=({o_x},{o_y})")



-P=(632001240931913206424725296092737555291996466411769017127257663531360159738505913372800853,1388990823665099828837937320113545375437773105670108733528204807940628993724679039963227531)
-P=(2,2)


## Zadanie 4
Zaimplementuj algorytm (funkcję), która oblicza P ⊕ Q sumę punktów krzywej eliptycznych. Zaimplementuj wszystkie przypadki. <br>
Dane: P = (x<sub>1</sub>, y<sub>1</sub>), Q = (x<sub>2</sub>, y<sub>2</sub>) ∈ E(F<sub>p</sub>) <br>
Wynik: R = (x<sub>3</sub>, y<sub>3</sub>) ∈ E(F<sub>p</sub>) taki, że R = P ⊕ Q.

In [65]:
from time import sleep
def eliptic_sum(x1, y1, x2, y2, p, A, B):
    x1_o, y1_o = opposite_point(x1, y1, p)

    if x2 == float('inf') and y2 == float('inf'):       # P + INF_P
        return x1, y1
    elif x1 == float('inf') and y1 == float('inf'):
        return x2, y2
    
    elif x2 ==x1_o and y2==y1_o:                        # P + -P
        return float('inf'), float('inf')
    
    elif x1 == x2 and y1 == y2:                         # P + P
        lam = ((mpz(3) * gmpy2.powmod(x1, mpz(2), p) + A) * (gmpy2.invert(2*y1, p))) % p
        x3 = (gmpy2.powmod(lam, mpz(2), p) - mpz(2) * x1) % p
        y3 = (lam * (x1 - x3) - y1) % p
        return x3, y3
    
    elif x1 != x2:                                       # P + Q
        lam = ((y2 - y1)*gmpy2.invert(x2-x1,p)) % p
        x3 = (gmpy2.powmod(lam, mpz(2), p) - x1 - x2) % p
        y3 = (lam * (x1-x3) - y1) % p
        return x3, y3


p = random_prime(k=300,p=3)
print(f"p={p}")
A, B = generate_random_curve(p=p)
x1, y1 = find_point_on_curve(A,B,p)
sleep(1)
x2, y2 = find_point_on_curve(A,B,p)
print(f"P=({x1},{y1})")
print(f"Q=({x2},{y2})")

x3,y3 = eliptic_sum(x1,y1,x2,y2,p,A,B)
print(f"P+Q=({x3},{y3})")






p=1209190243764830539897540748420729897686982584954007018549249398461167867377343101723591447
P=(1040525989921319378167680066565955445002440073598338531341959507715951342101836861851254046,197004223270094874291775984204358827077927911705015980377762613337901591705923998827355020)
Q=(779435042344436695024326097840543094124230939780408125040425066973770586780631710665806391,307553494233576374914534615688949409720223245980231378280937508417890967851534112622709380)
P+Q=(540591093681901591187119591351813715491888863845167070378083744632382334665578349735263663,277896869468123653535070796948174981171209212475221373455088364028521950930383996506703955)


## Zadanie 5
Zaimplementuj algorytm (funkcję), który oblicza n-tą wielokrotność punktu P.<br>
Dane: n ∈ N, P ∈ E(F<sub>p</sub>)<br>
Wynik: Q ∈ E(F<sub>p</sub>) taki, że Q = nP.<br>

Technika "Podwój i dodaj" analogiczna do efektywnego podnoszenia do kwadratu

In [97]:
def scalar_multiplication(n, x1, y1, A, B, p):
    Ox, Oy = float('inf'), float('inf')
    x, y = x1, y1

    while n > 0:
        if n % 2 == 1:
            Ox, Oy = eliptic_sum(Ox, Oy, x, y, p, A, B)
        x, y = eliptic_sum(x, y, x, y, p, A, B)
        n //= 2

    return Ox, Oy



n = 5
p = random_prime(k=300,p=3)
print(f"p={p}")
A, B = generate_random_curve(p=p)
x1, y1 = find_point_on_curve(A,B,p)

xn, yn = scalar_multiplication(n, x1, y1, A, B, p)
# xn, yn = scalar_multiplication(n, 3, 3, 1, 1, 7)

print(f"P=({x1},{y1})")
print(f"n={n}")
print(f"nP=({xn},{yn})")


p=1257896475140851593680918023379620807554261088390915376549793456869939852074954369509462151
P=(713566788109113703891523322524657565049480508300240159799384282597329714737394997979634012,375441489313881311108670752627745818285150784115529768510343994512730179502597533151416121)
n=5
nP=(214806276340487384830407709644292292362271910422854079320203430290945552557069488318231565,1064561919131684588123543490191362932202249396310847295790950953251695321304950921400334222)
